# CITS4403 - P2P Modelling

This notebook demonstrates a complex P2P model using agent gossip protocols (GNUTELLA-like) and graph-theory.

## Steps to Run
1. Run all cells sequentially.

In [1]:
import sys, os, pathlib
project_root = pathlib.Path.cwd().parent
sys.path.insert(0, str(project_root))
from src.graph import nxgraph
import src.agent as agent
from utils.plotter import draw_graph, draw_gossip_step_by_step, start_new_run, create_round_gif
#from utils.simulator_old import get_network_stats, reset_simulation
from utils.simulator_new import simulate_round_agent_driven, get_network_stats, reset_simulation
from widgets.graph_widget import display_graph_widgets, get_graph, is_graph_generated
from widgets.init_widget import display_init_widgets, set_graph_data
from widgets.scenario_widget import display_scenario_widgets
from widgets.simulation_widget import display_simulation_widgets

import ipywidgets as widgets
from IPython.display import display, clear_output


Below is the code for topologic comparison between BA, and ER graph

In [ ]:
# refactored code of above, storing data with dictionary
from src.graph import nxgraph
from utils.plotter import draw_graph
from src.analytics import clustering_co, path_length
import matplotlib.pyplot as plt

# Parameters, lower_ut = upper_ut effectively means graph is unweighted / uniformly weighted
node = 25
seed = 69
lower_ut = 50
upper_ut = 50
FILE_PIECES = 15
G = nxgraph()

# Unified storage
graph_data = {
    'BA': [],
    'ER': []
}

# Generate BA graphs and store metrics
for m in range(1, 5):
    ba = G.BA_graph(
        nodes=node,
        edges=m,
        seed=seed,
        weighted=True,
        lower_ut=lower_ut,
        upper_ut=upper_ut
    )

    avg_deg = sum(dict(ba.degree()).values()) / ba.number_of_nodes()
    cluster = round(clustering_co(ba), 2)
    path_L = round(path_length(ba), 2)
    edge_count = ba.number_of_edges()
    node_count = ba.number_of_nodes()

    graph_data['BA'].append({
        'graph': ba,
        'avg_degree': avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length': path_L,
        'num_nodes': node_count,
        'num_edges': edge_count
    })

    print(f"\nBA_graph (m={m})")
    draw_graph(ba, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"Average cluster coefficient: {cluster}")
    print(f"Average path length: {path_L}")
    print(f"Number of edges: {edge_count}")

# Generate ER graphs based on BA average degree
for ba_info in graph_data['BA']:
    avg_deg = ba_info['avg_degree']
    edge_count = int(node * avg_deg / 2)

    er = G.ER_Graph_nm(
        nodes=node,
        edges=edge_count,
        weighted=True,
        seed=seed,
        lower_ut=lower_ut,
        upper_ut=upper_ut
    )

    cluster = round(clustering_co(er), 2)
    path_L = round(path_length(er), 2)
    node_count = er.number_of_nodes()

    graph_data['ER'].append({
        'graph': er,
        'avg_degree': avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length': path_L,
        'num_nodes': node_count,
        'num_edges': edge_count
    })

    print(f"\nER_graph (matched to BA avg_deg={round(avg_deg,2)})")
    draw_graph(er, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"Average cluster coefficient: {cluster}")
    print(f"Average path length: {path_L}")
    print(f"Number of edges: {edge_count}")

print(graph_data['BA'])
def plot_metrics_vs_avg_degree(graph_data):
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle("BA vs ER Graph Metrics vs Average Degree", fontsize=16)

    # Plot Average Path Length vs Average Degree
    ax1 = axs[0]
    for kind, color, marker in [('BA', 'blue', 'o'), ('ER', 'green', 's')]:
        x = [rec['avg_degree'] for rec in graph_data[kind]]
        y = [rec['average_path_length'] for rec in graph_data[kind]]
        ax1.plot(x, y, marker=marker, linestyle='-', color=color, label=f"{kind}")
        for i, (xi, yi) in enumerate(zip(x, y)):
            ax1.text(xi, yi, str(i), fontsize=9, ha='right', va='bottom')
    ax1.set_xlabel("Average Degree")
    ax1.set_ylabel("Average Path Length")
    ax1.set_title("Path Length vs Degree")
    ax1.grid(True)
    ax1.legend()

    # Plot Clustering Coefficient vs Average Degree
    ax2 = axs[1]
    for kind, color, marker in [('BA', 'blue', 'o'), ('ER', 'green', 's')]:
        x = [rec['avg_degree'] for rec in graph_data[kind]]
        y = [rec['clustering_coefficient'] for rec in graph_data[kind]]
        ax2.plot(x, y, marker=marker, linestyle='-', color=color, label=f"{kind}")
        for i, (xi, yi) in enumerate(zip(x, y)):
            ax2.text(xi, yi, str(i), fontsize=9, ha='right', va='bottom')
    ax2.set_xlabel("Average Degree")
    ax2.set_ylabel("Clustering Coefficient")
    ax2.set_title("Clustering vs Degree")
    ax2.grid(True)
    ax2.legend()

    plt.tight_layout()
    plt.show()

plot_metrics_vs_avg_degree(graph_data)

Below is the code for scalability test

In [ ]:
# refactored code
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Scenario parameters
seed             = 69
edges_m          = 2
lower_ut         = 50
upper_ut         = 50
FILE_PIECES      = 15

n_seeders        = 1
sim_seed         = seed
search_mode      = 'realistic'
neighbor_selection = 'random'
ttl              = 5
cleanup_queries  = True
single_agent     = 0
save_images      = False
visualize_output = True
output_area      = widgets.Output()

G = nxgraph()

# Using a dictionary to store generated data
scal_test_graph_data = {
    'BA': [],
    'ER': []
}

# Generating BA graph
for n in range(10, 160, 20):
    ba = G.BA_graph(
        nodes    = n,
        edges    = edges_m,
        seed     = seed,
        weighted = True,
        lower_ut = lower_ut,
        upper_ut = upper_ut
    )

    node_count = ba.number_of_nodes()
    edge_count = ba.number_of_edges()
    avg_deg    = sum(dict(ba.degree()).values()) / node_count
    cluster    = round(clustering_co(ba), 2)
    path_L     = round(path_length(ba), 2)
    max_rounds = int(round(1.25 * n + 35, 0))

    scal_test_graph_data['BA'].append({
        'graph'                 : ba,
        'num_nodes'             : node_count,
        'num_edges'             : edge_count,
        'avg_degree'            : avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length'   : path_L,
        'max_rounds'            : max_rounds,
        'total_queries'         : 0,
        'total_hits'            : 0,
        'total_transfers'       : 0
    })

    print(f"\nBA_graph n={n}, m={edges_m}")
    draw_graph(ba, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"  | avg_deg={avg_deg:.2f}, C={cluster}, L={path_L}, edges={edge_count}")

# Generating ER graph
for ba_rec in scal_test_graph_data['BA']:
    n       = ba_rec['num_nodes']
    avg_deg = ba_rec['avg_degree']
    m_er    = int(n * avg_deg / 2)

    er = G.ER_Graph_nm(
        nodes    = n,
        edges    = m_er,
        weighted = True,
        seed     = seed,
        lower_ut = lower_ut,
        upper_ut = upper_ut
    )

    node_count = er.number_of_nodes()
    edge_count = er.number_of_edges()
    cluster    = round(clustering_co(er), 2)
    path_L     = round(path_length(er), 2)
    max_rounds = ba_rec['max_rounds']  # reuse same TTL logic

    scal_test_graph_data['ER'].append({
        'graph'                 : er,
        'num_nodes'             : node_count,
        'num_edges'             : edge_count,
        'avg_degree'            : avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length'   : path_L,
        'max_rounds'            : max_rounds,
        'total_queries'         : 0,
        'total_hits'            : 0,
        'total_transfers'       : 0
    })

    print(f"\nER_graph n={n}, m≈{m_er}")
    draw_graph(er, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"  | avg_deg={avg_deg:.2f}, C={cluster}, L={path_L}, edges={edge_count}")

# nested for loop for BA and ER
for kind in ('BA', 'ER'):
    for rec in scal_test_graph_data[kind]:
        Gk       = rec['graph']
        rounds   = rec['max_rounds']
        rec_q    = 0
        rec_h    = 0
        rec_t    = 0

        # initialise simulation
        set_graph_data(Gk, FILE_PIECES)
        agent.assign_n_seeders(Gk, n=n_seeders, seed=sim_seed)
        agent.initialize_file_sharing(
            Gk, FILE_PIECES,
            seed              = sim_seed,
            distribution_type = 'n_seeders',
            n_seeders         = n_seeders
        )

        # simulating
        for r in range(1, rounds + 1):
            result = simulate_round_agent_driven(
                Gk,
                FILE_PIECES,
                seed                    = sim_seed+r,
                cleanup_completed_queries=cleanup_queries,
                search_mode             = search_mode,
                current_round           = r,
                neighbor_selection      = neighbor_selection,
                single_agent            = (single_agent or None)
            )

            queries, hits, _ = result['message_rounds']
            transfers       = result['transfers']

            rec_q += len(queries)
            rec_h += len(hits)
            rec_t += len(transfers)
            stats = get_network_stats(Gk, FILE_PIECES)
            print(f"\n{kind} n={rec['num_nodes']} → Queries = {rec_q}, Hits = {rec_h}, Transfers = {rec_t}, Current round = {r}")
            print(f"Seeders: {stats['seeders']}")
            print(f"Leechers: {stats['leechers']} (incomplete: {stats['incomplete_leechers']})")
            print(f"Hybrids: {stats['hybrids']} (incomplete: {stats['incomplete_hybrids']})")
            print(f"Total pieces in network: {stats['total_pieces_in_network']}")
            print('about to go into stats')
            if stats['completion_rate'] >= 1.0:
                print(f"\nAll nodes have all pieces in {r} rounds")
                break

        rec['total_queries']   = rec_q
        rec['total_hits']      = rec_h
        rec['total_transfers'] = rec_t

        print(f"\n{kind} n={rec['num_nodes']} → Queries = {rec_q}, Hits = {rec_h}, Transfers = {rec_t}")
        
        draw_gossip_step_by_step(Gk, result['message_rounds'], result['transfers'], 
                            FILE_PIECES, r, save_images=False, 
                            max_ttl=ttl, show_debug_info=False)

# evaluation of results
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharex=True)

# Define metrics and styling
metrics = ['total_queries', 'total_hits', 'total_transfers']
titles  = ['Queries vs Nodes', 'Hits vs Nodes', 'Transfers vs Nodes']
colors  = {'BA': 'blue', 'ER': 'green'}
markers = {'BA': 'o', 'ER': 's'}

for i, metric in enumerate(metrics):
    ax = axes[i]
    for kind in ['BA', 'ER']:
        recs = scal_test_graph_data[kind]
        nodes = [r['num_nodes'] for r in recs]
        values = [r[metric] for r in recs]
        ax.plot(nodes, values, marker=markers[kind], label=kind, color=colors[kind])
    
    ax.set_title(titles[i])
    ax.set_xlabel("Number of Nodes")
    ax.set_ylabel("Count")
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()


The following code is testing weighted vs unweighted graphs. To run the code, simply run the very first cell, then run this cell.

In [ ]:
from utils.weighted_scenario import weighted_scene

weighted_test = weighted_scene(seed=69, 
                    nodes=30, 
                    lower_ut=25,
                    upper_ut=200,
                    FILE_PIECES=15,
                    n_seeders=1,
                    search_mode='realistic',
                    neighbor_selection='bandwith',
                    ttl=5,
                    single_agent=0,
                    weighted = True)

unweighted_test = weighted_scene(seed=42, 
                    nodes=30, 
                    lower_ut=50,
                    upper_ut=50,
                    FILE_PIECES=15,
                    n_seeders=1,
                    search_mode='realistic',
                    neighbor_selection='random',
                    ttl=5,
                    single_agent=0,
                    weighted = False)

#print('weighted vs unweighted: %s, %s' % (weighted_test['BA'][0]['final_round'],unweighted_test['BA'][0]['final_round']))
import matplotlib.pyplot as plt

# plot results

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

for ax, kind in zip(axes, ['BA', 'ER']):
    # Extract weighted data
    edges_w = [rec['num_edges']    for rec in weighted_test[kind]]
    fr_w    = [rec['final_round']  for rec in weighted_test[kind]]
    # Extract unweighted data
    edges_u = [rec['num_edges']    for rec in unweighted_test[kind]]
    fr_u    = [rec['final_round']  for rec in unweighted_test[kind]]

    ax.plot(edges_w, fr_w, marker='o', color='C0', label='Weighted')
    ax.plot(edges_u, fr_u, marker='s', color='C1', label='Unweighted')

    ax.set_title(f"{kind} Graph")
    ax.set_xlabel("Number of Edges")
    ax.set_ylabel("Final Round")
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

# plotting additional data
metrics = ['avg_degree', 'average_path_length', 'clustering_coefficient']
titles  = ['Average Degree vs Final Round',
           'Average Path Length vs Final Round',
           'Clustering Coefficient vs Final Round']
colors  = {'BA': 'blue', 'ER': 'green'}
markers = {'weighted': 'o', 'unweighted': 's'}

# Create subplots
metrics = ['avg_degree', 'average_path_length', 'clustering_coefficient']
titles  = ['Final Round vs Average Degree',
           'Final Round vs Average Path Length',
           'Final Round vs Clustering Coefficient']
colors  = {'BA': 'blue', 'ER': 'green'}
markers = {'weighted': 'o', 'unweighted': 's'}

# Create subplots
import matplotlib.pyplot as plt

def plot_structure_vs_final_round(weighted_test, unweighted_test):
    metrics = ['avg_degree', 'average_path_length', 'clustering_coefficient']
    titles  = ['Final Round vs Average Degree',
               'Final Round vs Average Path Length',
               'Final Round vs Clustering Coefficient']
    colors  = {'weighted': 'blue', 'unweighted': 'orange'}
    markers = {'weighted': 'o', 'unweighted': 's'}

    for kind in ['BA', 'ER']:
        fig, axs = plt.subplots(1, 3, figsize=(18, 5))
        fig.suptitle(f"{kind} Graphs: Structural Metrics vs Final Round", fontsize=16)

        for i, metric in enumerate(metrics):
            ax = axs[i]

            # Weighted
            x_w = [rec[metric] for rec in weighted_test[kind]]
            y_w = [rec['final_round'] for rec in weighted_test[kind]]
            ax.plot(x_w, y_w, marker=markers['weighted'], color=colors['weighted'],
                    label=f"{kind} Weighted")

            # Unweighted
            x_u = [rec[metric] for rec in unweighted_test[kind]]
            y_u = [rec['final_round'] for rec in unweighted_test[kind]]
            ax.plot(x_u, y_u, marker=markers['unweighted'], linestyle='--',
                    color=colors['unweighted'], label=f"{kind} Unweighted")

            ax.set_title(titles[i])
            ax.set_xlabel(metric.replace('_', ' ').title())
            ax.set_ylabel("Final Round")
            ax.legend()
            ax.grid(True)

        plt.tight_layout()
        plt.show()

plot_structure_vs_final_round(weighted_test, unweighted_test)

